In [1]:
import torch
from playgenie.model import VAE
from playgenie.loss import generative_loss

In [2]:
model  = VAE(input_size=782, hidden_size=500,latent_size=2)
sample_input  = torch.rand((10,1,782))
optim = torch.optim.Adam(model.parameters())

In [6]:
for epoch in range(100):
    xhat,mu,log_var = model(sample_input)
    optim.zero_grad()
    loss = generative_loss(x=sample_input,x_hat=xhat,mu= mu, log_var= log_var)
    loss.backward()
    optim.step()

    if epoch%10==0:
        print(f"Epoch:{epoch}:Loss:{loss.item()}")

Epoch:0:Loss:0.11480433493852615
Epoch:10:Loss:0.11465653032064438
Epoch:20:Loss:0.11450642347335815
Epoch:30:Loss:0.11422562599182129
Epoch:40:Loss:0.11435049772262573
Epoch:50:Loss:0.11482929438352585
Epoch:60:Loss:0.11463038623332977
Epoch:70:Loss:0.1146353930234909
Epoch:80:Loss:0.11473192274570465
Epoch:90:Loss:0.11482581496238708
